# Airbnb princing

In Airbnb the pricing should be set by the owner of the property. The suggestion is that the rate should be based on the monthly mortgage. 

In [37]:
import pandas as pd
import numpy as np

raw_data = pd.read_csv('data/Ames_Housing_Price_Data.csv', index_col=0)

rental_df = raw_data[['PID','SalePrice','YrSold']]
rental_df.sample(10)

,PID,SalePrice,YrSold
851,923276180,139000,2007
591,535378020,109500,2009
182,916402215,180000,2008
487,908102060,84900,2010
557,909252170,141500,2007
546,905452120,134500,2008
533,528180080,282500,2006
404,903234160,102000,2007
213,906380210,200000,2009
187,527401160,136000,2009


Group by OverallQual and get the mean and median SalePrice for each category

In [36]:
mortgage_df = pd.read_csv('data/MortageRates.csv')
mortgage_df['Average_30Year_Rate']= mortgage_df['Average_30Year_Rate'].str.replace('%','').astype(float)
mortgage_df.sample(10)

,Year,Average_30Year_Rate
4,1976,8.87
0,1972,7.38
13,1985,12.43
31,2003,5.83
40,2012,3.66
5,1977,8.85
23,1995,7.93
36,2008,6.03
6,1978,9.64
22,1994,8.38


In [38]:
rental_df = pd.merge(
    rental_df,
    mortgage_df,
    how="left",
    left_on='YrSold',
    right_on='Year',
    sort=True,
    suffixes=("_x", "_y"),
    copy=False).drop(columns = ['Year'])

rental_df.sample(10)

,PID,SalePrice,YrSold,Average_30Year_Rate
2326,528108120,376162,2010,4.69
345,535457070,146000,2006,6.41
2539,903426200,159900,2010,4.69
1669,903401050,145000,2008,6.03
566,528104010,345474,2007,6.34
2445,907401090,232000,2010,4.69
1470,527401130,133000,2008,6.03
2505,527402200,149000,2010,4.69
1339,535301080,116000,2008,6.03
2230,528142130,284000,2009,5.04


In [34]:
rental_df['YrSold'].value_counts()

2007    600
2009    585
2008    564
2006    516
2010    315
Name: YrSold, dtype: int64

According to Forbes, 2006 downpayment was 20%

Calculate monthly mortgage payment, not including taxes and insurance, using the following equation:

M = P [ i(1 + i)^n ] / [ (1 + i)^n – 1]

P = 0.8*SalePrice <- principal loan amount

i = monthly interest rate

n = 30*12 = 360

In [51]:
P = 0.8*rental_df['SalePrice']
i = rental_df['Average_30Year_Rate'].map(lambda x: x/(100*12))
n = 360
rental_df['MMortgage'] = P*((i*(1+i)**n))/(((1+i)**n)-1)
rental_df.sample(10)

,PID,SalePrice,YrSold,Average_30Year_Rate,MMortgage
1331,527165010,185000,2008,6.03,890.191350
913,909475230,315000,2007,6.34,1566.387996
2370,923228270,88250,2010,4.69,365.734070
2142,535402090,154000,2009,5.04,664.379297
933,921205070,281213,2007,6.34,1398.376722
360,527106010,222000,2006,6.41,1112.061618
1430,528280130,186500,2008,6.03,897.409118
700,907410050,213500,2007,6.34,1061.662975
1907,531371050,184900,2009,5.04,797.686571
832,908152290,171500,2007,6.34,852.811242
